## 15. **Pooling layer**

- 텐서의 특정 구역에서 특정 값(max, average)으로 대표값을 추출해 **차원을 줄이는 역할**을 함
- 연산량 감소, 과적합 방지, 중요한 정보 유지에 효과적
- Modern CNN
    - Pooling 대신 **stride가 있는 Convolution**을 사용함
        
        ⇒ Pooling은 아키텍처 상 **정보 손실**이 있고 stride를 통해서도 차원을 줄일 수 있음
        
    - **GAP (Global Average Pooling)**: Channel별로 전체 값을 평균내어 1개의 값으로 축소
        
        ⇒ 이미지의 위치에 관계없이 좋은 성능
        
        ⇒ 차원 축소 + **Flatten + Fully Connected Layer 생략 가능**
        
        ⇒ 바로 출력층과 연결 → 연산량 감소

In [7]:
import torch

# CPU/GPU 선택
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
import torch
import torch.nn as nn

input_tensor = torch.Tensor([[[[1, 2, 3, 4],
                             [5, 6, 7, 8],
                             [9, 10, 11, 12],
                             [13, 14, 15, 16],
                             ]]])

# Pooling layer 생성, 절반으로 줄이기
maxpool_layer = nn.MaxPool2d(kernel_size=2)
output_tensor = maxpool_layer(input_tensor)
print(f'Input Tensor shape: {input_tensor.shape}')
print(f'Output Tensor shape: {output_tensor.shape}')

Input Tensor shape: torch.Size([1, 1, 4, 4])
Output Tensor shape: torch.Size([1, 1, 2, 2])


In [9]:
import torch
from torchvision import datasets, transforms

# 이미지 변환(전처리)
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10은 10개의 클래스에 걸쳐 총 60,000개의 32x32 컬러 이미지로 구성된 데이터셋
train_dataset = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
)

test_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(test_dataset))



Files already downloaded and verified
Files already downloaded and verified
Train dataset size: 50000
Validation dataset size: 10000


In [ ]:


class ModernGAPCNN(nn.Module):
  def __init__(self, num_classes=10):
    super().__init__()

    self.feature_extractor = nn.Sequential(
      # 첫번째는 최대한 많은 정보를 담기 위해 stride=1로 설정
      nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=1), # 16 × 30 × 30
      nn.ReLU(inplace=True),
      # stride를 2로 설정하여 feature map 크기를 줄임(pooling 대체)
      nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=1), # 32 × 15 × 15
      nn.ReLU(inplace=True),
      nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=1), # 64 × 7 × 7
      nn.ReLU(inplace=True),
    )

    # Global Average Pooling을 이용해 Channel별로 전체 값을 평균내어 1개의 값으로 축소
    self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1)) # 64 × 1 × 1
    self.classifier = nn.Linear(64, num_classes)

  def forward(self, x):
      x = self.feature_extractor(x)
      x = self.global_avg_pool(x)
      x = torch.flatten(x, 1)
      x = self.classifier(x)
      return x


In [17]:
import torch.optim as optim

net = ModernGAPCNN(10)
net.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(10):
  net.train()
  for batch_idx, (data, label) in enumerate(train_loader):
    data, label = data.to(device), label.to(device)
    optimizer.zero_grad()
    output = net(data)
    train_loss = loss_fn(output, label)
    train_loss.backward()
    optimizer.step()

  net.eval()
  val_loss = 0
  correct = 0
  with torch.no_grad():
    for data, label in test_loader:
      data, label = data.to(device), label.to(device)
      output = net(data)
      val_loss += loss_fn(output, label).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(label.view_as(pred)).sum().item()
  
  val_loss /= len(test_loader.dataset)
  accuracy = 100. * correct / len(test_loader.dataset)

  print(f'Epoch {epoch+1}, Loss: {train_loss.item():.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')






Epoch 1, Loss: 1.1462, Val Loss: 0.3513, Accuracy: 49.58%
Epoch 2, Loss: 1.3162, Val Loss: 0.3160, Accuracy: 54.68%
Epoch 3, Loss: 1.9586, Val Loss: 0.2873, Accuracy: 60.09%
Epoch 4, Loss: 0.4086, Val Loss: 0.2747, Accuracy: 60.99%
Epoch 5, Loss: 0.2992, Val Loss: 0.2584, Accuracy: 63.62%
Epoch 6, Loss: 0.4884, Val Loss: 0.2623, Accuracy: 63.80%
Epoch 7, Loss: 0.7368, Val Loss: 0.2522, Accuracy: 65.15%
Epoch 8, Loss: 0.2222, Val Loss: 0.2531, Accuracy: 65.41%
Epoch 9, Loss: 0.3516, Val Loss: 0.2534, Accuracy: 65.73%
Epoch 10, Loss: 0.5158, Val Loss: 0.2457, Accuracy: 66.41%
